# 1. About the Question 1 in the Case Study

Question 1
Listening behavior on playlists is analyzed using the following tables:



● playlists table with columns:



    ○ user_id: the user who uploaded the playlist

    ○ playlist_id: the unique identifier of the playlist

● listings table with columns:

    ○ playlist_id: the unique identifier of the playlist

    ○ track_id: the unique identifier of the track

    ○ position: the position of this track in the playlist

    ○ track_duration: the length of the track recording (in milliseconds)


● plays table with columns:

    ○ timestamp: the time the play occurred

    ○ user_id: the user who played the given track

    ○ track_id: the unique identifier of the track

    ○ playlist_id: the unique identifier of the playlist (set to 0 if the play did not happen
    in a playlist)

    ○ listening_duration: the duration that the user listened to the track (in milliseconds)
    Using this schema, write or describe a SQL query that would help you find out the following:


##Questions

1. Given a user X, what are their 3 most listened to playlists?


2.  What is the percentage of users that create playlists only containing dj-sets? Assume a
dj-set is a track with a duration longer than 10min.


3.  For a user X, recommend 5 playlists they would like that they haven’t heard before. Go
for a practical over elaborate solution and stick to something that works well in SQL.
Please explain your logic.

# 2. Creating Temp Tables


Getting the data from the json file

In [ ]:
list_spec_listeners = df['listener_id'].unique()[1:30].tolist()

df_spec_listeners= df.query("listener_id in @list_spec_listeners")
df_spec_listeners.to_excel(excel_writer = "/Users/b.oeniz/Documents/df_spec_listeners.xlsx")

In [ ]:
SELECT  FROM `spry-precinct-256816.ben_spec.xx` 

## 2.1. Creating t_playlist table


The table that listeners upload their songs

``` sql
CREATE OR REPLACE TABLE spry-precinct-256816.ben_spec.t_playlist AS
WITH playlist AS (	
SELECT '3bd' AS playlist_id
UNION ALL	
SELECT '2d9' AS playlist_id
UNION ALL	
SELECT 'c47' AS playlist_id
UNION ALL	
SELECT 'f2c' AS playlist_id
UNION ALL	
SELECT 'c22' AS playlist_id
UNION ALL	
SELECT '76a' AS playlist_id	
UNION ALL	
SELECT '2b9' AS playlist_id	
UNION ALL	
SELECT '9e2' AS playlist_id
UNION ALL	
SELECT 'f32' AS playlist_id
UNION ALL	
SELECT 'e72' AS playlist_id
)	
SELECT *	
FROM playlist cross join 

(SELECT
  DISTINCT listener_id
FROM
  `spry-precinct-256816.ben_spec.xx`)
where RAND() <150/290
```


The table


``` sql
SELECT * FROM `spry-precinct-256816.ben_spec.t_playlist`

```


## 2.2. Creating t_listings table



``` sql
CREATE OR REPLACE TABLE spry-precinct-256816.ben_spec.t_listings AS
WITH t_playlist AS (	
SELECT '3bd' AS playlist_id
UNION ALL	
SELECT '2d9' AS playlist_id
UNION ALL	
SELECT 'c47' AS playlist_id
UNION ALL	
SELECT 'f2c' AS playlist_id
UNION ALL	
SELECT 'c22' AS playlist_id
UNION ALL	
SELECT '76a' AS playlist_id	
UNION ALL	
SELECT '2b9' AS playlist_id	
UNION ALL	
SELECT '9e2' AS playlist_id
UNION ALL	
SELECT 'f32' AS playlist_id
UNION ALL	
SELECT 'e72' AS playlist_id
)	
SELECT *	
FROM t_playlist cross join 

(SELECT
  DISTINCT track_id
,CAST( 1 + 10 * RAND() AS INT64)   as position
,track_duration
FROM
  `spry-precinct-256816.ben_spec.xx`)
where RAND() <600

```


The table


``` sql
SELECT * FROM `spry-precinct-256816.ben_spec.t_listings`

```



## 2.1. Creating t_plays table




```sql

CREATE OR REPLACE TABLE spry-precinct-256816.ben_spec.t_plays AS

SELECT   ts,
  listener_id,
  track_id,
  track_duration,
  listen_duration,
  playlist_id,
FROM
(SELECT 
  ts,
  tl.listener_id,
  track_id,
  track_duration,
  listen_duration,
  playlist_id,
   ROW_NUMBER() OVER(PARTITION BY tl.listener_id,track_id,playlist_id) AS rn,
   ROW_NUMBER() OVER(PARTITION BY tl.listener_id,track_id,playlist_id order by ts) AS rn_order

FROM
(SELECT
  ts,
  listener_id,
  track_id,
  track_duration,
  listen_duration
FROM
  `spry-precinct-256816.ben_spec.xx` 
WHERE listener_id IN ( select DISTINCT listener_id from `spry-precinct-256816.ben_spec.t_playlist` )
and track_id IN ( select DISTINCT track_id from `spry-precinct-256816.ben_spec.t_listings` )) as tl
LEFT JOIN 
(select * from `spry-precinct-256816.ben_spec.t_playlist` ) p
on tl.listener_id=p.listener_id)
WHERE rn in (2,5,10) OR rn_order in (3,8)

```
The table


``` sql
SELECT * FROM `spry-precinct-256816.ben_spec.t_plays`

```




# 3.Questions


## 3.1. Question 1
Given a user X, what are their 3 most listened to playlists?


NOTE: Queries are written in Google BigQuery



``` sql
SELECT
playlist_id
FROM
(SELECT
  listener_id,
  playlist_id,
  COUNT(*) AS number_of_times
FROM
  `spry-precinct-256816.ben_spec.t_plays`
WHERE playlist_id!=0 AND listener_id='000aefbc60a6f9a77371b39ed1ec54d3' --User X
GROUP BY
  1,
  2
ORDER BY
  number_of_times DESC
LIMIT
  3)


```



## 3.2. Question 2
What is the **percentage of users** that create playlists **only containing dj-sets**? Assume a dj-set is a track with duration longer than *10 min*.



``` sql
WITH t_users_with_non_dj AS (
SELECT
COUNT(DISTINCT listener_id) as number_of_listener_non_dj
FROM
  `spry-precinct-256816.ben_spec.t_playlist` p --user, playist 
INNER JOIN
  `spry-precinct-256816.ben_spec.t_listings` l
  ON p.playlist_id=l.playlist_id
WHERE track_duration < 600000 -- selecting the ones have at least one track less than 10 mins

), all_users AS (
SELECT
COUNT(DISTINCT listener_id) as number_of_users
FROM
  `spry-precinct-256816.ben_spec.t_playlist`
) 
SELECT  100- ROUND((SELECT number_of_listener_non_dj FROM t_users_with_non_dj)/(SELECT number_of_users FROM all_users)*100,2)

```



If a user creates a playlist but never add any track to it



``` sql
WITH t_users_with_non_dj AS (
SELECT
COUNT(DISTINCT listener_id) as number_of_listener_non_dj
FROM
  `spry-precinct-256816.ben_spec.t_playlist` p --maybe without track
LEFT JOIN
  `spry-precinct-256816.ben_spec.t_listings` l
  ON p.playlist_id=l.playlist_id
WHERE IFNULL(track_duration,0) < 600000 -- selecting the ones have at least one track less than 10 mins
---IFNULL, in case a playlist doesnt have any track

), all_users AS (
SELECT
COUNT(DISTINCT listener_id) as number_of_users
FROM
  `spry-precinct-256816.ben_spec.t_playlist`
) 
SELECT  100- ROUND((SELECT number_of_listener_non_dj FROM t_users_with_non_dj)/(SELECT number_of_users FROM all_users)*100,2)



```



## 3.3. Question 3
For a user X, **recommend 5 playlists** they would like that **they haven’t heard before**. 

Go for a practical over elaborate solution and stick to something that works well in SQL. Please explain your logic.

**Query 1:** Recommending the top playlists that listened by different users



```sql
SELECT 
playlist_id
FROM 

(SELECT
  playlist_id,
  COUNT(DISTINCT user_id) nr_times_playlist -- how many times a track listened by different users
FROM
  `dhh-digital-analytics-dwh-dev.bngs.plays`
WHERE
  playlist_id!='0' 
  AND playlist_id NOT IN (SELECT DISTINCT playlist_id FROM `dhh-digital-analytics-dwh-dev.bngs.plays` WHERE user_id='001a4db0d6d6adc9687c4f8ed2054c66')--User X
GROUP BY
  1
ORDER BY
  nr_times_playlist desc
LIMIT 5)


```



**Query 2:** Recommending the top playlists that listened the longest (Listen duration)



``` sql
SELECT 
playlist_id

FROM 
(SELECT
  playlist_id,
  SUM(listen_duration) total_listen_duration -- how long tracks listened 
FROM
  `dhh-digital-analytics-dwh-dev.bngs.plays`
WHERE
  playlist_id!='0' 
  AND playlist_id NOT IN (SELECT DISTINCT playlist_id FROM `dhh-digital-analytics-dwh-dev.bngs.plays` WHERE user_id='001a4db0d6d6adc9687c4f8ed2054c66')--User X
GROUP BY
  1
ORDER BY
  total_listen_duration desc
LIMIT 5)


```



**Query 3:** Recommending the top playlists that listened by the similar listeners

A=Playlists that User X listens 80% of the time 


B=Playlists that other users listens 80% of the time 

B/A > 0.5 Then recommned playlists that A NOT IN B



```sql

with users_listened_playlists as

(SELECT
      listener_id,
      playlist_id,
      SUM(listen_duration) sum_duration_playlist
    FROM
      `dhh-digital-analytics-dwh-dev.bngs.plays`
    GROUP BY
      listener_id,
      playlist_id

), users_playlists_ordered as
(SELECT
    listener_id,
    playlist_id,
    SUM(sum_duration_playlist) OVER (PARTITION BY listener_id  ORDER BY sum_duration_playlist DESC) cum_sum_track,
    SUM(sum_duration_playlist) OVER(PARTITION BY listener_id) AS total_sum,
    (SUM(sum_duration_playlist) OVER(PARTITION BY listener_id  ORDER BY sum_duration_playlist DESC)/ SUM(sum_duration_playlist) OVER(PARTITION BY listener_id)) AS percentage
  FROM users_listened_playlists 

), user_most_listened_playlists as
(SELECT
  listener_id,
  playlist_id
from users_playlists_ordered
WHERE
  percentage < 0.8) --playlists listened 80 % of time of the users --skipping the users who listened only 1 playlists


, t_similarity AS 
(SELECT
 listener_id as similar_users
 , count(DISTINCT playlist_id) as most_similar_playlists
 ,(SELECT COUNT(playlist_id) FROM user_most_listened_playlists where listener_id='001a4db0d6d6adc9687c4f8ed2054c66') AS x_most_listened
FROM  user_most_listened_playlists
WHERE playlist_id IN (SELECT playlist_id FROM user_most_listened_playlists where listener_id='001a4db0d6d6adc9687c4f8ed2054c66') --User X
AND listener_id!='001a4db0d6d6adc9687c4f8ed2054c66'
group by 1


), t_similarity_most AS 
(SELECT 
similar_users,(most_similar_playlists/x_most_listened) similarity_perc
FROM t_similarity

), not_listened_playlists AS 
(SELECT
  playlist_id,
  COUNT(DISTINCT listener_id) nr_times_playlist 
FROM
  user_most_listened_playlists
WHERE
  playlist_id!='0' 
  AND playlist_id NOT IN (SELECT DISTINCT playlist_id FROM `dhh-digital-analytics-dwh-dev.bngs.plays` WHERE listener_id='001a4db0d6d6adc9687c4f8ed2054c66')--User X
  AND  listener_id IN (SELECT similar_users FROM t_similarity_most WHERE similarity_perc> 0.5) --- similar users' most listened songs
GROUP BY
  1
ORDER BY
  nr_times_playlist desc
LIMIT 5)

SELECT playlist_id
FROM not_listened_playlists



```

